In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import plotly.graph_objects as go
from IPython.display import Image
import random

import warnings
warnings.filterwarnings("ignore")

In [2]:
data=pd.read_csv(r"C:\Users\Sunayana chittari\project\sales_train_evaluation.csv")
for i in range(1942,1942+28): #adding lastb 28 days as zero sales which we need to predicy it
    data['d_'+str(i)]=0

data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
calender=pd.read_csv(r"C:\Users\Sunayana chittari\project\calendar.csv")
calender=calender.fillna('no_event')

calender.head(4)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,no_event,no_event,no_event,no_event,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,no_event,no_event,no_event,no_event,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,no_event,no_event,no_event,no_event,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,no_event,no_event,no_event,no_event,1,1,0


# adding holiday and weekend feature

In [5]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

holidays=calendar().holidays(start=calender.date.min(), end=calender.date.max())

def is_holiday(x):
    if x in holidays:
        return 1
    else:
        return 0


weekends= ['Saturday', 'Sunday']
def if_weekends(x):
    if x in weekends:
        return 1
    else:
        return 0

In [6]:
calender['holiday']=calender['date'].apply(is_holiday)
calender['if_weekend']=calender['weekday'].apply(if_weekends)
calender.drop(['weekday','date'], axis=1, inplace=True)
calender.head()

,wm_yr_wk,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,holiday,if_weekend
0,11101,1,1,2011,d_1,no_event,no_event,no_event,no_event,0,0,0,0,1
1,11101,2,1,2011,d_2,no_event,no_event,no_event,no_event,0,0,0,0,1
2,11101,3,1,2011,d_3,no_event,no_event,no_event,no_event,0,0,0,0,0
3,11101,4,2,2011,d_4,no_event,no_event,no_event,no_event,1,1,0,0,0
4,11101,5,2,2011,d_5,no_event,no_event,no_event,no_event,1,0,1,0,0


In [7]:
sell_price=pd.read_csv(r"C:\Users\Sunayana chittari\project\sell_prices.csv")
sell_price.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [5]:
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == 'object':
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df  

data= downcast(data)
prices = downcast(sell_price)
calender = downcast(calender)

# melting and merging


In [8]:

#for each item making each row for sales happened on each day
#i.e making time series for day i.e day 1 to day 19..
def convert_single_sales_feature(input):
    melt = pd.melt(input, 
                id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
                var_name='d', value_name='sales').dropna()
    return melt

#merge iwth calendars
def merge_with_cal(input):
    value = pd.merge(input, calender, on='d', how='left')
    return value


#merging with price
def merge_with_price(input):
    value=pd.merge(input, prices, on=['store_id','item_id','wm_yr_wk'], how='left')
    return value


# melting and merging

In [ ]:
melt     =convert_single_sales_feature(data)
cal_merg =merge_with_cal(melt)
final    =merge_with_price(cal_merg)

In [9]:
def convert(input):
    return input.split("_")[1]
  
final['d']=final['d'].apply(convert)
final['d']=final['d'].astype('int16')

In [11]:
final[final['d']>1500].head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
45735000,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1501,0,2015-03-09,11506,...,3,2015,no_event,no_event,no_event,no_event,1,1,1,8.257812
45735001,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1501,1,2015-03-09,11506,...,3,2015,no_event,no_event,no_event,no_event,1,1,1,3.970703
45735002,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1501,0,2015-03-09,11506,...,3,2015,no_event,no_event,no_event,no_event,1,1,1,2.970703
45735003,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1501,1,2015-03-09,11506,...,3,2015,no_event,no_event,no_event,no_event,1,1,1,4.640625
45735004,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1501,1,2015-03-09,11506,...,3,2015,no_event,no_event,no_event,no_event,1,1,1,2.880859


In [17]:
s=final[final['d']>1500]
s.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
45735000,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1501,0,2015-03-09,11506,...,3,2015,no_event,no_event,no_event,no_event,1,1,1,8.257812
45735001,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1501,1,2015-03-09,11506,...,3,2015,no_event,no_event,no_event,no_event,1,1,1,3.970703
45735002,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1501,0,2015-03-09,11506,...,3,2015,no_event,no_event,no_event,no_event,1,1,1,2.970703
45735003,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1501,1,2015-03-09,11506,...,3,2015,no_event,no_event,no_event,no_event,1,1,1,4.640625
45735004,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1501,1,2015-03-09,11506,...,3,2015,no_event,no_event,no_event,no_event,1,1,1,2.880859


In [13]:
def convert(input):
    return input.split("_")[1]
  
final['d']=final['d'].apply(convert)
final['d']=final['d'].astype('int16')

# mean imputation

In [14]:
final['sell_price'].fillna(final.groupby(['store_id','item_id'])['sell_price'].transform('mean'),inplace=True)
final.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,wm_yr_wk,wday,...,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,holiday,if_weekend,sell_price
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1,0,11101,1,...,no_event,no_event,no_event,no_event,0,0,0,0,1,8.281250
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1,0,11101,1,...,no_event,no_event,no_event,no_event,0,0,0,0,1,3.970703
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1,0,11101,1,...,no_event,no_event,no_event,no_event,0,0,0,0,1,2.970703
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1,0,11101,1,...,no_event,no_event,no_event,no_event,0,0,0,0,1,4.527344
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1,0,11101,1,...,no_event,no_event,no_event,no_event,0,0,0,0,1,2.941406


In [15]:
for i in final.columns:
    if str(final[i].dtype)=='category':
        final[i] = final[i].cat.codes

In [51]:
final.to_csv(r"C:\Users\Sunayana chittari\project\all_data.csv")

# lag _faetures

In [16]:
lags = [28,30,35,42,49,56]
for lag in lags:
    final['sold_lag_'+str(lag)] = final.groupby(['item_id', 'store_id'],as_index=False)['sales'].shift(lag).astype(np.float16)


# #rolling_mean


In [15]:

final['rol_mean_7'] = final.groupby(['item_id','store_id'])['sales'].transform(lambda x: x.rolling(window=7).mean()).astype(np.float16)
final['rol_mean_14'] = final.groupby(['item_id','store_id'])['sales'].transform(lambda x: x.rolling(window=14).mean()).astype(np.float16)
final['rol_mean_28'] = final.groupby(['item_id','store_id'])['sales'].transform(lambda x: x.rolling(window=28).mean()).astype(np.float16)

# #sales based features

In [17]:


def mean_sold(input,target):
    final[target]=final.groupby(input)['sales'].transform('mean').astype(np.float16)

mean_sold('item_id','item_sold_avg')
mean_sold('store_id','avg_sold_storewis')
mean_sold('state_id','avg_sold_statewis')
mean_sold('cat_id','avg_sold_catwis')
mean_sold(['item_id','store_id'],'avg_sold_store_item')
mean_sold(['item_id','state_id'],'avg_sold_state_item')

#'item_sold_avg','avg_sold_storewis','avg_sold_statewis','avg_sold_catwis','avg_sold_store_item','avg_sold_state_item',

# price based features

In [73]:



def mean_price(input,target):
    final[target]=final.groupby(input)['sell_price'].transform('mean').astype(np.float16)


mean_price('item_id','item_price_avg')
mean_price('store_id','avg_pric_storewis')
mean_price('state_id','avg_pri_statewis')
mean_price('cat_id','avg_price_catwis')
mean_price(['item_id','store_id'],'avg_price_store_item')
mean_price(['item_id','state_id'],'avg_price_state_item')

'item_sold_avg','avg_sold_storewis','avg_sold_statewis','avg_sold_catwis','avg_sold_store_item','avg_sold_state_item','item_price_avg','avg_pric_storewis','avg_pri_statewis','avg_price_catwis','avg_price_store_item','avg_price_state_item'
